# Graph Summarizer

In [1]:
import networkx as nx
import random
import numpy as np
import nltk
from nltk.cluster.util import cosine_distance

try:
    from nltk.corpus import stopwords
except:
    import nltk
    nltk.download('stopwords')
finally:
    from nltk.corpus import stopwords

## Load Text

In [2]:
try:
    book = nltk.corpus.gutenberg.raw('melville-moby_dick.txt')
except:
    nltk.download('gutenberg')
finally:
    book = nltk.corpus.gutenberg.raw('melville-moby_dick.txt')

## Clean Text

In [3]:
def read_article(data):
    article = data.split('. ')
    sentences = []
    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()
    return sentences

## Create Network

In [4]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

## Summarize

In [ ]:
%%time
data = book
stop_words = stopwords.words('english')
summarize_text = []

# Step 1 - Read text anc split it
print("Step 1")
sentences =  read_article(data)

# Step 2 - Generate Similary Martix across sentences
print("Step 2")
sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

# Step 3 - Rank sentences in similarity martix
print("Step 3")
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)

Step 1
Step 2


In [ ]:
print(nx.info(sentence_similarity_graph))

In [5]:
def generate_summary(data, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    print("Step 1")
    sentences =  read_article(data)

    # Step 2 - Generate Similary Martix across sentences
    print("Step 2")
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    print("Step 3")
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    print("Step 4")
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
#     print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Step 5")
    return ". ".join(summarize_text)

In [6]:
%time generate_summary(book, top_n=5)

KeyboardInterrupt: 

---